## Imports

In [1]:
# pip install --upgrade scikit-learn

In [2]:

import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.3
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [3]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "C:\\Users\\Win10\\Desktop\\titanic\\"

## Load data

In [4]:
df      = pd.read_csv(DATA_PATH + "train.csv.txt", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv.txt",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [7]:
df['Name'].head(10)

PassengerId
1                               Braund, Mr. Owen Harris
2     Cumings, Mrs. John Bradley (Florence Briggs Th...
3                                Heikkinen, Miss. Laina
4          Futrelle, Mrs. Jacques Heath (Lily May Peel)
5                              Allen, Mr. William Henry
6                                      Moran, Mr. James
7                               McCarthy, Mr. Timothy J
8                        Palsson, Master. Gosta Leonard
9     Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
10                  Nasser, Mrs. Nicholas (Adele Achem)
Name: Name, dtype: object

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [8]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()




df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)



df['Title'].unique(), df_test['Title'].unique()

(array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
        'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
        'Jonkheer'], dtype=object),
 array(['Mr', 'Mrs', 'Miss', 'Master', 'Ms', 'Col', 'Rev', 'Dr', 'Dona'],
       dtype=object))

In [9]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [10]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [11]:
# Use map to apply the prevous dict

df['Title']      = df.Title.map(title_dictionary)
df_test['Title'] = df_test.Title.map(title_dictionary)



In [12]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [13]:
df_test['Ticket'].isna().sum()/len(df)

0.0

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [14]:
df_test['Cabin'].isna().sum()/len(df_test)

0.7822966507177034

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [15]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [16]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [17]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
"""
num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))]) # mean, median

cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder(handle_unknown='ignore'))])
tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),],
    remainder='drop')
                                     
tree_prepro



ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [18]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [19]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [23]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":HistGradientBoostingClassifier(),
  "XGBoost":XGBClassifier(use_label_encoder=False),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers['LightGBM']

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [24]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [25]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
# YOUR CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size = 0.2,
    stratify=y,
    random_state=42)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train, y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

[17:00:59] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6868404	total: 160ms	remaining: 2m 39s
1:	learn: 0.6805085	total: 165ms	remaining: 1m 22s
2:	learn: 0.6751704	total: 168ms	remaining: 55.8s
3:	learn: 0.6703740	total: 172ms	remaining: 42.7s
4:	learn: 0.6656806	total: 175ms	remaining: 34.9s
5:	learn: 0.6606047	total: 178ms	remaining: 29.5s
6:	learn: 0.6556054	total: 181ms	remaining: 25.7s
7:	learn: 0.6509279	total: 184ms	remaining: 22.8s
8:	learn: 0.6457662	total: 191ms	remaining: 21.1s
9:	learn: 0.6405683	total: 196ms	remaining: 19.4s
10:	learn: 0.6354775	total: 199ms	remaining: 17.9s
11:	learn: 0.6303684	total: 203ms	remaining: 16.7s
12:	learn: 0.6251300	total: 206ms	remaining: 15.6s
13:	learn: 0.6202156	total: 209ms	remaining: 14.7s
14:	learn: 0

198:	learn: 0.3787948	total: 816ms	remaining: 3.29s
199:	learn: 0.3783835	total: 821ms	remaining: 3.28s
200:	learn: 0.3779339	total: 825ms	remaining: 3.28s
201:	learn: 0.3775900	total: 829ms	remaining: 3.27s
202:	learn: 0.3773614	total: 832ms	remaining: 3.26s
203:	learn: 0.3771293	total: 836ms	remaining: 3.26s
204:	learn: 0.3766560	total: 859ms	remaining: 3.33s
205:	learn: 0.3765137	total: 863ms	remaining: 3.33s
206:	learn: 0.3764347	total: 866ms	remaining: 3.31s
207:	learn: 0.3759590	total: 869ms	remaining: 3.31s
208:	learn: 0.3754416	total: 873ms	remaining: 3.3s
209:	learn: 0.3749490	total: 879ms	remaining: 3.31s
210:	learn: 0.3746032	total: 883ms	remaining: 3.3s
211:	learn: 0.3743302	total: 893ms	remaining: 3.32s
212:	learn: 0.3740313	total: 898ms	remaining: 3.32s
213:	learn: 0.3734776	total: 902ms	remaining: 3.31s
214:	learn: 0.3730706	total: 905ms	remaining: 3.31s
215:	learn: 0.3725882	total: 909ms	remaining: 3.3s
216:	learn: 0.3721357	total: 913ms	remaining: 3.29s
217:	learn: 0.3

357:	learn: 0.3417560	total: 1.63s	remaining: 2.92s
358:	learn: 0.3415845	total: 1.63s	remaining: 2.91s
359:	learn: 0.3413380	total: 1.64s	remaining: 2.91s
360:	learn: 0.3412744	total: 1.64s	remaining: 2.9s
361:	learn: 0.3410846	total: 1.64s	remaining: 2.9s
362:	learn: 0.3410009	total: 1.65s	remaining: 2.89s
363:	learn: 0.3407696	total: 1.66s	remaining: 2.89s
364:	learn: 0.3405932	total: 1.66s	remaining: 2.88s
365:	learn: 0.3404380	total: 1.66s	remaining: 2.88s
366:	learn: 0.3401547	total: 1.67s	remaining: 2.87s
367:	learn: 0.3400150	total: 1.67s	remaining: 2.87s
368:	learn: 0.3399025	total: 1.68s	remaining: 2.86s
369:	learn: 0.3397311	total: 1.68s	remaining: 2.86s
370:	learn: 0.3394099	total: 1.68s	remaining: 2.85s
371:	learn: 0.3391136	total: 1.69s	remaining: 2.85s
372:	learn: 0.3389099	total: 1.69s	remaining: 2.84s
373:	learn: 0.3387544	total: 1.69s	remaining: 2.83s
374:	learn: 0.3386288	total: 1.7s	remaining: 2.83s
375:	learn: 0.3385039	total: 1.7s	remaining: 2.82s
376:	learn: 0.33

517:	learn: 0.3189322	total: 2.23s	remaining: 2.08s
518:	learn: 0.3188865	total: 2.24s	remaining: 2.08s
519:	learn: 0.3187656	total: 2.24s	remaining: 2.07s
520:	learn: 0.3187447	total: 2.25s	remaining: 2.07s
521:	learn: 0.3186261	total: 2.25s	remaining: 2.06s
522:	learn: 0.3183688	total: 2.27s	remaining: 2.07s
523:	learn: 0.3181594	total: 2.27s	remaining: 2.06s
524:	learn: 0.3180445	total: 2.28s	remaining: 2.06s
525:	learn: 0.3179035	total: 2.29s	remaining: 2.06s
526:	learn: 0.3177923	total: 2.29s	remaining: 2.06s
527:	learn: 0.3177298	total: 2.3s	remaining: 2.05s
528:	learn: 0.3177254	total: 2.3s	remaining: 2.05s
529:	learn: 0.3175623	total: 2.31s	remaining: 2.04s
530:	learn: 0.3173872	total: 2.31s	remaining: 2.04s
531:	learn: 0.3172084	total: 2.31s	remaining: 2.04s
532:	learn: 0.3170974	total: 2.32s	remaining: 2.03s
533:	learn: 0.3170222	total: 2.32s	remaining: 2.02s
534:	learn: 0.3169791	total: 2.32s	remaining: 2.02s
535:	learn: 0.3168437	total: 2.33s	remaining: 2.01s
536:	learn: 0.

677:	learn: 0.3002115	total: 3.06s	remaining: 1.45s
678:	learn: 0.3001348	total: 3.07s	remaining: 1.45s
679:	learn: 0.3001262	total: 3.07s	remaining: 1.44s
680:	learn: 0.3000424	total: 3.09s	remaining: 1.45s
681:	learn: 0.2999551	total: 3.1s	remaining: 1.44s
682:	learn: 0.2998774	total: 3.11s	remaining: 1.44s
683:	learn: 0.2996961	total: 3.11s	remaining: 1.44s
684:	learn: 0.2996352	total: 3.12s	remaining: 1.43s
685:	learn: 0.2995975	total: 3.12s	remaining: 1.43s
686:	learn: 0.2995169	total: 3.12s	remaining: 1.42s
687:	learn: 0.2994911	total: 3.13s	remaining: 1.42s
688:	learn: 0.2993333	total: 3.14s	remaining: 1.42s
689:	learn: 0.2992396	total: 3.14s	remaining: 1.41s
690:	learn: 0.2991419	total: 3.15s	remaining: 1.41s
691:	learn: 0.2991123	total: 3.15s	remaining: 1.4s
692:	learn: 0.2989974	total: 3.16s	remaining: 1.4s
693:	learn: 0.2989788	total: 3.16s	remaining: 1.39s
694:	learn: 0.2987800	total: 3.17s	remaining: 1.39s
695:	learn: 0.2986676	total: 3.17s	remaining: 1.39s
696:	learn: 0.2

863:	learn: 0.2809193	total: 4.47s	remaining: 704ms
864:	learn: 0.2808568	total: 4.48s	remaining: 699ms
865:	learn: 0.2807549	total: 4.48s	remaining: 694ms
866:	learn: 0.2806373	total: 4.49s	remaining: 690ms
867:	learn: 0.2806122	total: 4.5s	remaining: 684ms
868:	learn: 0.2805322	total: 4.51s	remaining: 680ms
869:	learn: 0.2803618	total: 4.51s	remaining: 674ms
870:	learn: 0.2802807	total: 4.52s	remaining: 669ms
871:	learn: 0.2802261	total: 4.52s	remaining: 664ms
872:	learn: 0.2801124	total: 4.54s	remaining: 661ms
873:	learn: 0.2798815	total: 4.55s	remaining: 656ms
874:	learn: 0.2798017	total: 4.55s	remaining: 651ms
875:	learn: 0.2797875	total: 4.56s	remaining: 645ms
876:	learn: 0.2796419	total: 4.57s	remaining: 640ms
877:	learn: 0.2794986	total: 4.57s	remaining: 635ms
878:	learn: 0.2794322	total: 4.58s	remaining: 630ms
879:	learn: 0.2792921	total: 4.58s	remaining: 625ms
880:	learn: 0.2792829	total: 4.59s	remaining: 620ms
881:	learn: 0.2790145	total: 4.59s	remaining: 614ms
882:	learn: 0

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,84.357542,82.140975,6.324263
2,Skl HistGBM,82.681564,80.777339,0.637177
3,Random Forest,82.122905,80.592885,0.388094
4,Decision Tree,81.564246,80.408432,0.059550
5,Extra Trees,81.005587,79.953887,0.366087
6,Skl GBM,81.005587,78.333333,0.325600
7,LightGBM,80.446927,78.959157,0.221560
8,AdaBoost,78.770950,78.135705,0.235572
9,XGBoost,78.770950,77.865613,0.166552


In [26]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [27]:
"""
skf = model_selection.StratifiedKFold(
    # CODE HERE

"""
# YOUR CODE HERE
skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

print(x.shape)
print(y.shape)
for model_name, model in tree_classifiers.items():
    start_time = time.time()

    pred = model_selection.cross_val_predict(model, x, y,cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)




results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


(891, 8)
(891,)
[17:01:53] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:01:53] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:01:54] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:01:54] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to re

141:	learn: 0.4035814	total: 492ms	remaining: 2.97s
142:	learn: 0.4028758	total: 497ms	remaining: 2.98s
143:	learn: 0.4024078	total: 501ms	remaining: 2.98s
144:	learn: 0.4018920	total: 505ms	remaining: 2.98s
145:	learn: 0.4013138	total: 508ms	remaining: 2.97s
146:	learn: 0.4007712	total: 513ms	remaining: 2.97s
147:	learn: 0.4006029	total: 515ms	remaining: 2.96s
148:	learn: 0.4000807	total: 522ms	remaining: 2.98s
149:	learn: 0.3998508	total: 526ms	remaining: 2.98s
150:	learn: 0.3992978	total: 529ms	remaining: 2.97s
151:	learn: 0.3990466	total: 532ms	remaining: 2.97s
152:	learn: 0.3984901	total: 535ms	remaining: 2.96s
153:	learn: 0.3981162	total: 538ms	remaining: 2.96s
154:	learn: 0.3975556	total: 541ms	remaining: 2.95s
155:	learn: 0.3971245	total: 544ms	remaining: 2.94s
156:	learn: 0.3966860	total: 549ms	remaining: 2.95s
157:	learn: 0.3963110	total: 551ms	remaining: 2.94s
158:	learn: 0.3961491	total: 553ms	remaining: 2.93s
159:	learn: 0.3957350	total: 556ms	remaining: 2.92s
160:	learn: 

335:	learn: 0.3547078	total: 1.14s	remaining: 2.24s
336:	learn: 0.3545022	total: 1.14s	remaining: 2.24s
337:	learn: 0.3543746	total: 1.14s	remaining: 2.24s
338:	learn: 0.3541569	total: 1.15s	remaining: 2.24s
339:	learn: 0.3539908	total: 1.15s	remaining: 2.24s
340:	learn: 0.3537125	total: 1.16s	remaining: 2.23s
341:	learn: 0.3536742	total: 1.16s	remaining: 2.23s
342:	learn: 0.3536294	total: 1.16s	remaining: 2.23s
343:	learn: 0.3535011	total: 1.17s	remaining: 2.23s
344:	learn: 0.3533339	total: 1.17s	remaining: 2.22s
345:	learn: 0.3532642	total: 1.17s	remaining: 2.21s
346:	learn: 0.3530754	total: 1.18s	remaining: 2.21s
347:	learn: 0.3530020	total: 1.18s	remaining: 2.21s
348:	learn: 0.3528625	total: 1.18s	remaining: 2.2s
349:	learn: 0.3525673	total: 1.18s	remaining: 2.2s
350:	learn: 0.3524536	total: 1.19s	remaining: 2.2s
351:	learn: 0.3524457	total: 1.19s	remaining: 2.19s
352:	learn: 0.3522616	total: 1.19s	remaining: 2.19s
353:	learn: 0.3520349	total: 1.2s	remaining: 2.18s
354:	learn: 0.35

539:	learn: 0.3280975	total: 1.8s	remaining: 1.54s
540:	learn: 0.3280870	total: 1.81s	remaining: 1.53s
541:	learn: 0.3280864	total: 1.81s	remaining: 1.53s
542:	learn: 0.3280027	total: 1.83s	remaining: 1.54s
543:	learn: 0.3279083	total: 1.83s	remaining: 1.53s
544:	learn: 0.3278259	total: 1.83s	remaining: 1.53s
545:	learn: 0.3277549	total: 1.84s	remaining: 1.53s
546:	learn: 0.3276087	total: 1.85s	remaining: 1.53s
547:	learn: 0.3274880	total: 1.85s	remaining: 1.53s
548:	learn: 0.3273745	total: 1.86s	remaining: 1.52s
549:	learn: 0.3270621	total: 1.86s	remaining: 1.52s
550:	learn: 0.3268704	total: 1.88s	remaining: 1.53s
551:	learn: 0.3268087	total: 1.88s	remaining: 1.53s
552:	learn: 0.3266267	total: 1.89s	remaining: 1.52s
553:	learn: 0.3265398	total: 1.89s	remaining: 1.52s
554:	learn: 0.3264750	total: 1.89s	remaining: 1.52s
555:	learn: 0.3264510	total: 1.9s	remaining: 1.51s
556:	learn: 0.3263811	total: 1.9s	remaining: 1.51s
557:	learn: 0.3262346	total: 1.9s	remaining: 1.51s
558:	learn: 0.32

731:	learn: 0.3082210	total: 2.67s	remaining: 977ms
732:	learn: 0.3079814	total: 2.67s	remaining: 973ms
733:	learn: 0.3079717	total: 2.67s	remaining: 970ms
734:	learn: 0.3078545	total: 2.68s	remaining: 966ms
735:	learn: 0.3076773	total: 2.68s	remaining: 962ms
736:	learn: 0.3075939	total: 2.69s	remaining: 960ms
737:	learn: 0.3074413	total: 2.69s	remaining: 956ms
738:	learn: 0.3073148	total: 2.7s	remaining: 954ms
739:	learn: 0.3071921	total: 2.71s	remaining: 951ms
740:	learn: 0.3071315	total: 2.71s	remaining: 947ms
741:	learn: 0.3070529	total: 2.71s	remaining: 943ms
742:	learn: 0.3069814	total: 2.72s	remaining: 940ms
743:	learn: 0.3069070	total: 2.72s	remaining: 936ms
744:	learn: 0.3068414	total: 2.72s	remaining: 932ms
745:	learn: 0.3067121	total: 2.73s	remaining: 928ms
746:	learn: 0.3066097	total: 2.73s	remaining: 924ms
747:	learn: 0.3065518	total: 2.73s	remaining: 920ms
748:	learn: 0.3064863	total: 2.73s	remaining: 917ms
749:	learn: 0.3064020	total: 2.74s	remaining: 913ms
750:	learn: 0

...

482:	learn: 0.3283187	total: 8.68s	remaining: 9.29s
483:	learn: 0.3281955	total: 8.69s	remaining: 9.26s
484:	learn: 0.3280202	total: 8.69s	remaining: 9.23s
485:	learn: 0.3280051	total: 8.7s	remaining: 9.2s
486:	learn: 0.3277988	total: 8.7s	remaining: 9.17s
487:	learn: 0.3276940	total: 8.73s	remaining: 9.15s
488:	learn: 0.3276240	total: 8.75s	remaining: 9.14s
489:	learn: 0.3274448	total: 8.76s	remaining: 9.11s
490:	learn: 0.3272611	total: 8.76s	remaining: 9.09s
491:	learn: 0.3272076	total: 8.77s	remaining: 9.05s
492:	learn: 0.3271614	total: 8.78s	remaining: 9.03s
493:	learn: 0.3270766	total: 8.86s	remaining: 9.07s
494:	learn: 0.3267699	total: 8.86s	remaining: 9.04s
495:	learn: 0.3266892	total: 8.87s	remaining: 9.01s
496:	learn: 0.3266189	total: 8.87s	remaining: 8.98s
497:	learn: 0.3265833	total: 8.88s	remaining: 8.95s
498:	learn: 0.3265562	total: 8.89s	remaining: 8.92s
499:	learn: 0.3264439	total: 8.89s	remaining: 8.89s
500:	learn: 0.3263028	total: 8.94s	remaining: 8.9s
501:	learn: 0.32

642:	learn: 0.3116062	total: 11.1s	remaining: 6.19s
643:	learn: 0.3113876	total: 11.2s	remaining: 6.17s
644:	learn: 0.3112179	total: 11.2s	remaining: 6.14s
645:	learn: 0.3110470	total: 11.2s	remaining: 6.13s
646:	learn: 0.3109715	total: 11.2s	remaining: 6.11s
647:	learn: 0.3108489	total: 11.2s	remaining: 6.09s
648:	learn: 0.3107509	total: 11.2s	remaining: 6.07s
649:	learn: 0.3106230	total: 11.3s	remaining: 6.08s
650:	learn: 0.3104266	total: 11.3s	remaining: 6.06s
651:	learn: 0.3104053	total: 11.3s	remaining: 6.04s
652:	learn: 0.3103119	total: 11.3s	remaining: 6.01s
653:	learn: 0.3101421	total: 11.3s	remaining: 5.99s
654:	learn: 0.3099761	total: 11.3s	remaining: 5.97s
655:	learn: 0.3099593	total: 11.3s	remaining: 5.95s
656:	learn: 0.3098470	total: 11.4s	remaining: 5.96s
657:	learn: 0.3097811	total: 11.4s	remaining: 5.94s
658:	learn: 0.3097223	total: 11.4s	remaining: 5.92s
659:	learn: 0.3096983	total: 11.4s	remaining: 5.9s
660:	learn: 0.3096146	total: 11.5s	remaining: 5.87s
661:	learn: 0

802:	learn: 0.2946358	total: 14.5s	remaining: 3.56s
803:	learn: 0.2945597	total: 14.5s	remaining: 3.54s
804:	learn: 0.2945206	total: 14.5s	remaining: 3.52s
805:	learn: 0.2944983	total: 14.5s	remaining: 3.5s
806:	learn: 0.2944271	total: 14.5s	remaining: 3.48s
807:	learn: 0.2943812	total: 14.5s	remaining: 3.46s
808:	learn: 0.2943529	total: 14.6s	remaining: 3.45s
809:	learn: 0.2943038	total: 14.6s	remaining: 3.43s
810:	learn: 0.2940525	total: 14.6s	remaining: 3.41s
811:	learn: 0.2939986	total: 14.6s	remaining: 3.39s
812:	learn: 0.2938704	total: 14.6s	remaining: 3.37s
813:	learn: 0.2937308	total: 14.6s	remaining: 3.34s
814:	learn: 0.2935218	total: 14.6s	remaining: 3.32s
815:	learn: 0.2934132	total: 14.7s	remaining: 3.3s
816:	learn: 0.2933393	total: 14.7s	remaining: 3.28s
817:	learn: 0.2932668	total: 14.7s	remaining: 3.26s
818:	learn: 0.2931540	total: 14.7s	remaining: 3.25s
819:	learn: 0.2930727	total: 14.7s	remaining: 3.23s
820:	learn: 0.2930363	total: 14.7s	remaining: 3.21s
821:	learn: 0.

961:	learn: 0.2799022	total: 17.2s	remaining: 680ms
962:	learn: 0.2798390	total: 17.2s	remaining: 661ms
963:	learn: 0.2796070	total: 17.2s	remaining: 643ms
964:	learn: 0.2794294	total: 17.2s	remaining: 625ms
965:	learn: 0.2792982	total: 17.3s	remaining: 607ms
966:	learn: 0.2792634	total: 17.5s	remaining: 596ms
967:	learn: 0.2791684	total: 17.5s	remaining: 578ms
968:	learn: 0.2791388	total: 17.5s	remaining: 559ms
969:	learn: 0.2790742	total: 17.5s	remaining: 541ms
970:	learn: 0.2789985	total: 17.5s	remaining: 523ms
971:	learn: 0.2789259	total: 17.5s	remaining: 504ms
972:	learn: 0.2788787	total: 17.5s	remaining: 486ms
973:	learn: 0.2787786	total: 17.6s	remaining: 470ms
974:	learn: 0.2786624	total: 17.6s	remaining: 451ms
975:	learn: 0.2786393	total: 17.6s	remaining: 433ms
976:	learn: 0.2783685	total: 17.6s	remaining: 415ms
977:	learn: 0.2783103	total: 17.6s	remaining: 396ms
978:	learn: 0.2781534	total: 17.6s	remaining: 378ms
979:	learn: 0.2781032	total: 17.7s	remaining: 361ms
980:	learn: 

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,83.277217,81.027706,3.219795
2,CatBoost,83.277217,81.248202,97.097744
3,Skl HistGBM,82.491582,80.831176,9.931062
4,LightGBM,82.491582,80.886300,1.210255
5,Random Forest,81.930415,80.541175,4.570769
6,XGBoost,81.930415,80.430927,3.446942
7,AdaBoost,81.144781,79.903653,3.583864
8,Extra Trees,80.695847,79.318857,3.549533
9,Decision Tree,79.349046,78.060589,0.454541


In [30]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [31]:

best_model = tree_classifiers["Skl GBM"].fit(x,y)
# Fit best model with all data



# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [33]:
# test_pred = # Get the predictions for x_test
test_pred = best_model.predict(x_test)


In [34]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [35]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [36]:
sub.to_csv("sub.csv")

In [37]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
